In [1]:
import pandas as pd
from pymongo import MongoClient
from configparser import ConfigParser
# from plotly.express 

/home/ben/.virtualenvs/cosc545-oPe5D360/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
config = ConfigParser()
config.read("creds.cfg")
